In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
from glob import glob
import scipy.signal as signal

In [2]:

def filter_timeseries(ts, order, cutoff, btype='lowpass', fs=1, **kwargs):
    """Given an array, this function apply a butterworth (high/low pass) 
    filter of the given order and cutoff frequency.
    For example:
    If 'ts' is a timeseries of daily samples, filter_timeseries(ts,3,1/20)
    will return the series without the 20 days or less variability using an
    order 3 butterworth filter. 
    In the same way, filter_timeseries(ts,3,1/20, btype='highpass') will
    return the series with only the 20 days or less variability.

    Args:
        ts (array_like): timeseries or 1D array to filter
        order (int): _description_
        cutoff (array_like): Single float for lowpass or highpass filters, 
        arraylike for bandpass filters.
        btype (str, optional): The type of filter. Defaults to 'lowpass'.
        fs (int): Sampling frequency. Defaults to 1
        **kwargs are passed to scipy.signal.filtfilt

    Returns:
        output (array): Filtered array
    """
    mask = np.isnan(ts)
    nans = np.ones(len(ts))*np.nan
    if mask.sum()==len(ts):
        return nans
    else:
        b, a = signal.butter(order, cutoff, btype=btype, fs=fs)
        filt=signal.filtfilt(b, a, ts[~mask], padlen=30*max(len(a),len(b)), **kwargs)
        output=np.ones(len(ts))*np.nan
        output[np.where(~mask)] = filt
        return output
    
def filter_xarray(data, dim, order, cutoff, btype='lowpass', parallel=False, fs=1):
    """Given a 3d DataArray, with time and spatial coordinates, this function apply
    the 1D function filter_timeseries along the time dimension, filter the complete
    xarray data.

    Args:
        data (XDataArray): data
        dim (str): name of the time dimension
        order (int): butterworth filter order
        cutoff (array_like): if float, the cutoff frequency, if array must be the
                            [min,max] frequencys for the bandpass filter.
        btype (str, optional): {lowpass,highpass,bandpass}. Defaults to 'lowpass'.
        parallel (bool, optional): If parallelize with dask. Defaults to False.
        fs (int, optional): Sampling frequency. Defaults to 1.

    Returns:
        XDataArray: filtered data
    """
    if parallel:
        dask='parallelized'
    else:
        dask='forbidden'
    filt = xr.apply_ufunc(filter_timeseries, data, order, cutoff, btype, fs,
                          input_core_dims=[[dim],[],[],[],[]],
                          output_core_dims=[[dim]],
                          exclude_dims=set((dim,)),
                          keep_attrs=True,
                          vectorize=True, dask=dask)
    filt[dim] = data[dim]
    return filt



In [3]:
tropical_glorys       = xr.open_dataset('data/GLORYS12V1/HOVMOLLERS/glorys_tropical.nc').swap_dims({'lon':'index'}).convert_calendar('noleap', dim='time')
tropical_reforecast   = xr.open_dataset('data/S2S/HOVMOLLERS/reforecasts_tropical.nc').swap_dims({'lon':'index'}).convert_calendar('noleap', dim='inittime')

coastnorth_glorys     = xr.open_dataset('data/GLORYS12V1/HOVMOLLERS/glorys_coastnorth.nc').swap_dims({'lat':'index'}).convert_calendar('noleap', dim='time')
coastnorth_reforecast = xr.open_dataset('data/S2S/HOVMOLLERS/reforecasts_coastnorth.nc').swap_dims({'lat':'index'}).convert_calendar('noleap', dim='inittime')

coastsouth_glorys     = xr.open_dataset('data/GLORYS12V1/HOVMOLLERS/glorys_coastsouth.nc').swap_dims({'lat':'index'}).convert_calendar('noleap', dim='time')
coastsouth_reforecast = xr.open_dataset('data/S2S/HOVMOLLERS/reforecasts_coastsouth.nc').swap_dims({'lat':'index'}).convert_calendar('noleap', dim='inittime')

In [7]:

filter_xarray(tropical_glorys.groupby('time.dayofyear').mean(),'dayofyear',5,1/60).to_netcdf('/home/lucas/storage/glorys_tropical.nc')
filter_xarray(coastnorth_glorys.groupby('time.dayofyear').mean(),'dayofyear',5,1/60).to_netcdf('/home/lucas/storage/glorys_coastnorth.nc')
filter_xarray(coastsouth_glorys.groupby('time.dayofyear').mean(),'dayofyear',5,1/60).to_netcdf('/home/lucas/storage/glorys_coastsouth.nc')

In [ ]:
tropical_reforecast_clim   = []
coastnorth_reforecast_clim = []
coastsouth_reforecast_clim = []
for lead in range(len(tropical_reforecast.leadtime)):
    data = tropical_reforecast.sel(leadtime=lead)
    data.coords['time'] = ('inittime', data.inittime.values+pd.Timedelta(days=lead))
    data = data.swap_dims({'inittime':'time'}).drop('inittime').groupby('time.dayofyear').mean()
    data = data.interp({'dayofyear':np.arange(1,366)}).ffill('dayofyear').bfill('dayofyear')
    data = filter_xarray(data, 'dayofyear', 5, 1/60)
    tropical_reforecast_clim.append(data)
    
    data = coastnorth_reforecast.sel(leadtime=lead)
    data.coords['time'] = ('inittime', data.inittime.values+pd.Timedelta(days=lead))
    data = data.swap_dims({'inittime':'time'}).drop('inittime').groupby('time.dayofyear').mean()
    data = data.interp({'dayofyear':np.arange(1,366)}).ffill('dayofyear').bfill('dayofyear')
    data = filter_xarray(data, 'dayofyear', 5, 1/60)
    coastnorth_reforecast_clim.append(data)
    
    data = coastsouth_reforecast.sel(leadtime=lead)
    data.coords['time'] = ('inittime', data.inittime.values+pd.Timedelta(days=lead))
    data = data.swap_dims({'inittime':'time'}).drop('inittime').groupby('time.dayofyear').mean()
    data = data.interp({'dayofyear':np.arange(1,366)}).ffill('dayofyear').bfill('dayofyear')
    data = filter_xarray(data, 'dayofyear', 5, 1/60)
    coastsouth_reforecast_clim.append(data)
    
tropical_reforecast_clim   = xr.concat(tropical_reforecast_clim, 'leadtime')
coastnorth_reforecast_clim = xr.concat(coastnorth_reforecast_clim, 'leadtime')
coastsouth_reforecast_clim = xr.concat(coastsouth_reforecast_clim, 'leadtime')

In [ ]:
coastsouth_reforecast_clim.to_netcdf('data/S2S/CLIMATOLOGY/reforecasts_coastsouth_clim.nc')
coastnorth_reforecast_clim.to_netcdf('data/S2S/CLIMATOLOGY/reforecasts_coastnorth_clim.nc')
tropical_reforecast_clim.to_netcdf('data/S2S/CLIMATOLOGY/reforecasts_tropical_clim.nc')